## Paquets i dades originals ## 

In [649]:
import json
import pandas as pd
from datetime import datetime

from auxiliary_functions import (obtenir_data_presencia_codi, restar_dates, codis_ICD, nombre_ingressos,
                                 dies_ingressat_total, interval_10_edat,
                                 sumar_barthel, sumar_emina, obtenir_ultima_clau, obtenir_pes_o_mitjana,
                                 canadenca_comparada, disfagia_mecvvs, extreure_valors_claus,
                                 extreure_valors_claus_simple, obtenir_valors_clau_interes, index_charlson,
                                 obtenir_pes_mes_antic, obtenir_pes_mes_nou, obtenir_data_pes_mes_antic,
                                 obtenir_primera_data_mecvv, obtenir_pes_coincident_mecvv, restar_columnes_object)
from listas import (PA_list, P_list, disfagia_list, Main_respiratory_infections_list, LRTI_list,
                    COPD_exacerbations_list,
                    Pulmonary_fibrosis_fibrotorax_list, priorfalls_list, delirium_list, dementia_list, depresyndr_list,
                    uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list,
                    sarcopenia_list,
                    sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list,
                    ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list,
                    psicosis_list, nutridef_list, charlson_dict)

taula = pd.read_pickle('./data/processed/dataframe.pkl')
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [650]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,None,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,None,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Distribuiré les dades en 3 grups (més un grup que conté la totalitat de dades):
- **AMB_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **AMB_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Han sigut diagnosticats amb el codi ICD de pneumònia
    - Han passat menys de 30 dies entre el diagnòstic amb l'ICD de pneumònia i el resultat positiu al MECVV
- **SENSE_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han passat més de 30 dies entre el diagnòstic amb l'ICD de pneumònia i  el resultat positiu del MECVV 
- **TOTAL**: aquest grup conté tots els pacients de l'estudi *#TODO: hacer descriptiva de este grupo*

# AMB_PA 

In [651]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,None,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,2021-08-19,None,None,None,2021-08-17,2.0
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,46.4,46.4,2020-09-09,2020-09-07,46.4,0.0,0.0,2020-09-04,3.0


# AMB_PA_MECVV

In [652]:
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)] 
AMB_PA_MECVV.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,2021-08-19,None,None,None,2021-08-17,2.0
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,46.4,46.4,2020-09-09,2020-09-07,46.4,0.0,0.0,2020-09-04,3.0
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,None,41.100,0,2021-01-20,2020-11-25,None,None,41.1,2020-12-02,7.0
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,None,0,0,2022-05-30,2022-05-29,0,0.0,0.0,2022-05-23,6.0


# SENSE_PA

In [653]:
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) & 
           (taula['P diagnosticada'] == 1.0)] 
SENSE_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,None,None,None,None,2020-09-07,None,None,None,2022-02-14,525.0
25,83,F,0,"[{'date': '20220308', 'name': 'UREA Sèrum', 'l...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '0', 'estatMental': '1', 'humit...",[],"[{'valor': '57', 'data': '2022-11-03'}, {'valo...","[{'codi': 'C10AA', 'dataInici': '2022-09-27 00...","[{'data': '20220929091303', 'disfagia': 'S', '...",...,None,57,63.2,2022-11-03,2022-09-29,None,None,-6.2,2023-04-09,192.0
31,89,M,0,"[{'date': '20220318', 'name': 'BILIRUBINA TOTA...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '75', 'data': '2022-01-11'}, {'valo...","[{'codi': 'C10AA', 'dataInici': '2021-12-23 00...","[{'data': '20220119101523', 'disfagia': 'N', '...",...,None,75,67,2022-01-11,2022-02-01,None,None,8.0,2023-05-05,458.0
39,87,F,0,"[{'date': '20230619', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2021-01-17 00...","[{'data': '20220111085147', 'disfagia': 'S', '...",...,None,None,None,None,2022-01-11,None,None,None,2023-06-19,524.0
41,93,F,0,"[{'date': '20221013', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '0', 'data': '2023-06-02'}, {'valor...","[{'codi': 'C10AA', 'dataInici': '2021-02-05 00...","[{'data': '20230530155654', 'disfagia': 'S', '...",...,None,0,60,2023-06-02,2023-05-30,0,0.0,-60.0,2023-03-21,70.0


# TOTAL

In [654]:
TOTAL = taula
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,None,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,None,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Ara que tinc totes les dades classificades en aquests grups, faré una descriptiva. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat per mirar la seva distribució, la qual esperaré que sigui similar entre el grup AMB_PA i AMB_PA_MECVV, i diferent entre SENSE_PA i els altres 2 grups. Els criteris que seguiré per utilitzar un test o un altre és, primer, fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Nombre de pacients

In [655]:
num_AMB_PA = len(AMB_PA)
num_AMB_PA_MECVV= len(AMB_PA_MECVV)
num_SENSE_PA = len(SENSE_PA)
num_TOTAL = len(TOTAL)

print(num_AMB_PA, num_AMB_PA_MECVV, num_SENSE_PA, num_TOTAL)

17 8 8 50


#### Pvalor

##### Normalitat

In [656]:
stat, p_value = shapiro(range(num_AMB_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9621
Valor p = 0.6704
El nombre de files segueix una distribució normal.


In [657]:
stat, p_value = shapiro(range(num_AMB_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9749
Valor p = 0.9332
El nombre de files segueix una distribució normal.


In [658]:
stat, p_value = shapiro(range(num_SENSE_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9749
Valor p = 0.9332
El nombre de files segueix una distribució normal.


In [659]:
stat, p_value = shapiro(range(num_TOTAL))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9556
Valor p = 0.0581
El nombre de files segueix una distribució normal.


In [660]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [661]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA i SENSE_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i SENSE_PA: 0.0338
No existeixen diferències significatives entre els 2 grups.


In [662]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i SENSE_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [663]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [664]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i TOTAL: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [665]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA_MECVV i TOTAL: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [666]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre SENSE_PA i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre SENSE_PA i TOTAL: 0.7033
No existeixen diferències significatives entre els 2 grups.


## Edat

In [667]:
edad_AMB_PA = AMB_PA['edat'].mean()
desviacio_estandard_edad = AMB_PA['edat'].std()

resultado = f"{edad_AMB_PA:.2f} ± {desviacio_estandard_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [668]:
edad_AMB_PA_MECVV = AMB_PA_MECVV['edat'].mean()
desviacio_estandard_edad = AMB_PA_MECVV['edat'].std()

resultado = f"{edad_AMB_PA_MECVV:.2f} ± {desviacio_estandard_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 91.50 ± 6.82


In [669]:
edad_SENSE_PA = SENSE_PA['edat'].mean()
desviacio_estandard_edad = SENSE_PA['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacio_estandard_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 86.25 ± 6.90


In [670]:
edad_TOTAL = TOTAL['edat'].mean()
desviacio_estandard_TOTAL = TOTAL['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacio_estandard_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 86.25 ± 6.90


#### Pvalor

##### Normalitat 

In [671]:
stat, p_value = shapiro(AMB_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El nombre de files segueix una distribució normal.


In [672]:
stat, p_value = shapiro(AMB_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8304
Valor p = 0.0600
El nombre de files segueix una distribució normal.


In [673]:
stat, p_value = shapiro(SENSE_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8318
Valor p = 0.0619
El nombre de files segueix una distribució normal.


In [674]:
stat, p_value = shapiro(TOTAL['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8491
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [675]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['edat'], AMB_PA_MECVV['edat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.18896133816771837


In [676]:
stat, p_value_13 = mannwhitneyu(AMB_PA['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9534289321662349


In [677]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.37026415517950617


In [678]:
stat, p_value_14 = mannwhitneyu(AMB_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.20417514607442822


In [679]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.05636751107260842


In [680]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.4769108663506575


## Sexe

In [681]:
sexe_AMB_PA = AMB_PA['sexe'].value_counts()
sexe_AMB_PA_MECVV = AMB_PA_MECVV['sexe'].value_counts()
sexe_SENSE_PA = SENSE_PA['sexe'].value_counts()
sexe_TOTAL = TOTAL['sexe'].value_counts()

In [682]:
if 'F' in sexe_AMB_PA:
    percent_female1 = (sexe_AMB_PA['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA")

Percentatge de sexe femení en AMB_PA: 47.06%


In [683]:
if 'F' in sexe_AMB_PA_MECVV:
    percent_female2 = (sexe_AMB_PA_MECVV['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA_MECVV")

Percentatge de sexe femení en AMB_PA_MECVV: 47.06%


In [684]:
if 'F' in sexe_SENSE_PA:
    percent_female3 = (sexe_SENSE_PA['F'] / len(SENSE_PA)) * 100
    print(f"Percentatge de sexe femení en SENSE_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SENSE_PA")


Percentatge de sexe femení en SENSE_PA: 47.06%


In [685]:
if 'F' in sexe_TOTAL:
    percent_female3 = (sexe_TOTAL['F'] / len(TOTAL)) * 100
    print(f"Percentatge de sexe femení en TOTAL: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en TOTAL")

Percentatge de sexe femení en TOTAL: 47.06%


#### Pvalor

##### Normalitat

In [686]:
#TODO: ???Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [687]:
contingency_table_AMB_PA_AMB_PA_MECVV = pd.crosstab(index=AMB_PA['sexe'], columns=AMB_PA_MECVV['sexe'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_AMB_PA_AMB_PA_MECVV)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [688]:
contingency_table_AMB_PA_SENSE_PA = pd.crosstab(index=AMB_PA['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [689]:
contingency_table_AMB_PA_MECVV_SENSE_PA = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_MECVV_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [690]:
contingency_table_14 = pd.crosstab(index=AMB_PA['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.


In [691]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.


In [692]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['sexe'], columns=TOTAL['sexe'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.5
P-valor entre SENSE_PA i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.


## Ingressos (admissions)

In [693]:
Mitjana_ingresos = AMB_PA['Nombre ingressos'].mean()
desviacio_estandard_ingresos = AMB_PA['Nombre ingressos'].std()

resultado = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [694]:
Mitjana_ingresos = AMB_PA_MECVV['Nombre ingressos'].mean()
desviacio_estandard_ingresos = AMB_PA_MECVV['Nombre ingressos'].std()

resultado = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.88 ± 1.64


In [695]:
Mitjana_ingresos = SENSE_PA['Nombre ingressos'].mean()
desviacio_estandard_ingresos = SENSE_PA['Nombre ingressos'].std()

resultado = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 8.75 ± 3.85


In [696]:
Mitjana_ingresos = TOTAL['Nombre ingressos'].mean()
desviacio_estandard_ingresos = TOTAL['Nombre ingressos'].std()

resultado = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 6.68 ± 5.08


#### Pvalor

##### Normalitat

In [697]:
stat, p_value = shapiro(AMB_PA['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El nombre de files no segueix una distribució normal.


In [698]:
stat, p_value = shapiro(AMB_PA_MECVV['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9122
Valor p = 0.3695
El nombre de files segueix una distribució normal.


In [699]:
stat, p_value = shapiro(SENSE_PA['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9498
Valor p = 0.7091
El nombre de files segueix una distribució normal.


In [700]:
stat, p_value = shapiro(TOTAL['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7256
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [701]:
stat, p_value = shapiro(TOTAL['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7256
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [702]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Nombre ingressos'], AMB_PA_MECVV['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.27403132109149697


In [703]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Nombre ingressos'], SENSE_PA['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.042821773602008555


In [704]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Nombre ingressos'], SENSE_PA['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.18434809692120802


In [705]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.4941763107233279


In [706]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.6902658476564807


In [707]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.06577367478634001


## Disfàgia

In [708]:
disf_AMB_PA= (AMB_PA['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [709]:
disf_AMB_PA_MECVV = (AMB_PA_MECVV['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA_MECVV:.2f}%")

Percentatge de disfàgia: 50.00%


In [710]:
disf_SENSE_PA= (SENSE_PA['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SENSE_PA:.2f}%")

Percentatge de disfàgia: 25.00%


In [711]:
disf_TOTAL= (TOTAL['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_TOTAL:.2f}%")

Percentatge de disfàgia: 20.00%


#### Pvalor

##### Normalitat

In [712]:
stat, p_value = shapiro(AMB_PA['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [713]:
stat, p_value = shapiro(AMB_PA_MECVV['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [714]:
stat, p_value = shapiro(SENSE_PA['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [715]:
stat, p_value = shapiro(TOTAL['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [716]:
contingency_table_12 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=AMB_PA_MECVV['DO diagnosticada'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.


In [717]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=SENSE_PA['DO diagnosticada'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.


In [718]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DO diagnosticada'], columns=SENSE_PA['DO diagnosticada'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [719]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.


In [720]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.


In [721]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.


## Index de Barthel

### Mitjana

In [722]:
Mitjana_barthel = AMB_PA['Barthel resultats'].mean()
desviacio_estandard_barthel = AMB_PA['Barthel resultats'].std()

resultado = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [723]:
Mitjana_barthel = AMB_PA_MECVV['Barthel resultats'].mean()
desviacio_estandard_barthel = AMB_PA_MECVV['Barthel resultats'].std()

resultado = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 13.75 ± 18.27


In [724]:
Mitjana_barthel = SENSE_PA['Barthel resultats'].mean()
desviacio_estandard_barthel = SENSE_PA['Barthel resultats'].std()

resultado = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 53.12 ± 21.20


#### Pvalor

##### Normalitat

In [725]:
stat, p_value = shapiro(AMB_PA['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El nombre de files no segueix una distribució normal.


In [726]:
stat, p_value = shapiro(AMB_PA_MECVV['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8000
Valor p = 0.0286
El nombre de files no segueix una distribució normal.


In [727]:
stat, p_value = shapiro(SENSE_PA['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9475
Valor p = 0.6859
El nombre de files segueix una distribució normal.


In [728]:
stat, p_value = shapiro(TOTAL['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8866
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [969]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'], AMB_PA_MECVV['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.47528505928431597


In [971]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'], SENSE_PA['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

P-valor entre AMB_PA i SENSE_PA: 0.00906817282777616


In [972]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'], SENSE_PA['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_13)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.00906817282777616


In [974]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_value_13)

P-valor entre AMB_PA i TOTAL: 0.00906817282777616


In [975]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL: 0.015133604112910346


In [977]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

P-valor entre SENSE_PA i TOTAL: 0.6668885476033475


### Valors de dependència

##### AMB_PA

In [735]:
indepe_AMB_PA = (AMB_PA['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA: {indepe_AMB_PA:.2f}%")

Percentatge de indepe_AMB_PA: 0.00%


In [736]:
mod_AMB_PA = (AMB_PA['Barthel resultats'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_AMB_PA: {mod_AMB_PA:.2f}%")

Percentatge de mod_AMB_PA: 5.88%


In [737]:
sev_AMB_PA = (AMB_PA['Barthel resultats'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_AMB_PA: {sev_AMB_PA:.2f}%")

Percentatge de sev_AMB_PA: 41.18%


In [738]:
total_AMB_PA = (AMB_PA['Barthel resultats'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_AMB_PA: {total_AMB_PA:.2f}%")

Percentatge de total_AMB_PA: 52.94%


##### AMB_PA_MECVV

In [739]:
indepe_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA_MECVV: {indepe_AMB_PA_MECVV:.2f}%")

Percentatge de indepe_AMB_PA_MECVV: 0.00%


In [740]:
mod_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'].between(61,95)).mean() * 100
print(f"Percentatge de mod_AMB_PA_MECVV: {mod_AMB_PA_MECVV:.2f}%")

Percentatge de mod_AMB_PA_MECVV: 0.00%


In [741]:
sev_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'].between(21,60)).mean() * 100
print(f"Percentatge de sev_AMB_PA_MECVV: {sev_AMB_PA_MECVV:.2f}%")

Percentatge de sev_AMB_PA_MECVV: 25.00%


In [742]:
total_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'] <= 20.0).mean() * 100
print(f"Percentatge de total_AMB_PA_MECVV: {total_AMB_PA_MECVV:.2f}%")

Percentatge de total_AMB_PA_MECVV: 75.00%


##### SENSE_PA

In [743]:
indepe_SENSE_PA = (SENSE_PA['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SENSE_PA: {indepe_SENSE_PA:.2f}%")

Percentatge de indepe_SENSE_PA: 0.00%


In [744]:
mod_SENSE_PA = (SENSE_PA['Barthel resultats'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SENSE_PA: {mod_SENSE_PA:.2f}%")

Percentatge de mod_SENSE_PA: 37.50%


In [745]:
sev_SENSE_PA = (SENSE_PA['Barthel resultats'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SENSE_PA: {sev_SENSE_PA:.2f}%")

Percentatge de sev_SENSE_PA: 62.50%


In [746]:
total_SENSE_PA = (SENSE_PA['Barthel resultats'] <= 20).mean() * 100
print(f"Percentatge de total_SENSE_PA: {total_SENSE_PA:.2f}%")

Percentatge de total_SENSE_PA: 0.00%


#### Pvalor

##### Independència

In [747]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'] == 100.0, AMB_PA_MECVV['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV:", p_value_12)

P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV: 1.0


In [748]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'] == 100.0, SENSE_PA['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i indepe_SENSE_PA: 1.0


In [749]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Barthel resultats'] == 100.0, SENSE_PA['Barthel resultats'] == 100.0, equal_var=False)
print("P-valor entre indepe_AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre indepe_AMB_PA_MECVV i SENSE_PA: 0.18434809692120802


In [978]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'] == 100.0, TOTAL['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_TOTAL:", p_value_14)

P-valor entre indepe_AMB_PA i indepe_TOTAL: 0.04855526544534558


In [979]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'] == 100.0, TOTAL['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA_MECVV i indepe_TOTAL:", p_value_24)

P-valor entre indepe_AMB_PA_MECVV i indepe_TOTAL: 0.17345845133549298


In [980]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'] == 100.0, TOTAL['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_SENSSE_PA i indepe_TOTAL:", p_value_34)

P-valor entre indepe_SENSSE_PA i indepe_TOTAL: 0.17345845133549298


##### Moderada

In [750]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'].between(61, 95), AMB_PA_MECVV['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV:", p_value_12)

P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV: 0.5483429354628137


In [751]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'].between(61, 95), SENSE_PA['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_AMB_PA i mod_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i mod_SENSE_PA: 0.05422099630917637


In [752]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(61, 95), SENSE_PA['Barthel resultats'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA :", p_value_23)

P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA : 0.07458641569139825


In [981]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'].between(61, 95), TOTAL['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_TOTAL:", p_value_14)

P-valor entre mod_AMB_PA i mod_TOTAL: 0.23385439959965149


In [982]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(61, 95), TOTAL['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA_MECVV i mod_TOTAL:", p_value_24)

P-valor entre mod_AMB_PA_MECVV i mod_TOTAL: 0.20185601492817817


In [983]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'].between(61, 95), TOTAL['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_SENSE_PA i mod_TOTAL:", p_value_34)

P-valor entre mod_SENSE_PA i mod_TOTAL: 0.21602478204322784


##### Severa

In [753]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'].between(21,60), AMB_PA_MECVV['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV:", p_value_12)

P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV: 0.4622499461255427


In [754]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'].between(21,60), SENSE_PA['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_SENSE_PA:", p_value_13)

P-valor entre sev_AMB_PA i sev_SENSE_PA: 0.3463247915940385


In [755]:
t_stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(21,60), SENSE_PA['Barthel resultats'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA:", p_value_23)

P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA: 0.16065967802771053


In [984]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'].between(21,60), TOTAL['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_TOTAL:", p_value_14)

P-valor entre sev_AMB_PA i sev_TOTAL: 0.31965398550389235


In [985]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(21,60), TOTAL['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA_MECVV i sev_TOTAL:", p_value_24)

P-valor entre sev_AMB_PA_MECVV i sev_TOTAL: 0.8727371942854353


In [986]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'].between(21,60), TOTAL['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_SENSE_PA i sev_TOTAL:", p_value_34)

P-valor entre sev_SENSE_PA i sev_TOTAL: 0.05745304713254242


##### Total

In [756]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'] <= 20.0, AMB_PA_MECVV['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_AMB_PA_MECVV:", p_value_12)

P-valor entre total_AMB_PA i total_AMB_PA_MECVV: 0.31988618684011016


In [757]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'] <= 20.0, SENSE_PA['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_SENSE_PA:", p_value_13)

P-valor entre total_AMB_PA i total_SENSE_PA: 0.01293706233933995


In [758]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'] <= 20.0, SENSE_PA['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_SENSE_PA:", p_value_23)

P-valor entre total_AMB_PA_MECVV i total_SENSE_PA: 0.0033087017190950086


In [987]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'] <= 20.0, TOTAL['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_TOTAL:", p_value_14)

P-valor entre total_AMB_PA i total_TOTAL: 0.17208392420429652


In [988]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'] <= 20.0, TOTAL['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_TOTAL:", p_value_24)

P-valor entre total_AMB_PA_MECVV i total_TOTAL: 0.030106371453454194


In [989]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'] <= 20.0, TOTAL['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_SENSE_PA i total_TOTAL:", p_value_34)

P-valor entre total_SENSE_PA i total_TOTAL: 0.0535696765403014


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [991]:
AMB_PA.loc[:, 'MNA resultats'] = pd.to_numeric(AMB_PA['MNA resultats'], errors='coerce')
AMB_PA_MECVV.loc[:, 'MNA resultats'] = pd.to_numeric(AMB_PA_MECVV['MNA resultats'], errors='coerce')
SENSE_PA.loc[:, 'MNA resultats'] = pd.to_numeric(SENSE_PA['MNA resultats'], errors='coerce')
TOTAL.loc[:, 'MNA resultats'] = pd.to_numeric(TOTAL['MNA resultats'], errors='coerce')

### Mitjana

In [760]:
Mitjana_mna = AMB_PA['MNA resultats'].mean()
desviacio_estandard_mna = AMB_PA['MNA resultats'].std()

resultado = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [761]:
Mitjana_mna = AMB_PA_MECVV['MNA resultats'].mean()
desviacio_estandard_mna = AMB_PA_MECVV['MNA resultats'].std()

resultado = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [762]:
Mitjana_mna = SENSE_PA['MNA resultats'].mean()
desviacio_estandard_mna = SENSE_PA['MNA resultats'].std()

resultado = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 0.00 ± 0.00


In [992]:
Mitjana_mna = TOTAL['MNA resultats'].mean()
desviacio_estandard_mna = TOTAL['MNA resultats'].std()

resultado = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 4.41 ± 7.20


#### Pvalor

##### Normalitat

In [763]:
stat, p_value = shapiro(AMB_PA['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [764]:
stat, p_value = shapiro(AMB_PA_MECVV['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [765]:
stat, p_value = shapiro(SENSE_PA['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [993]:
stat, p_value = shapiro(TOTAL['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [766]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['MNA resultats'], AMB_PA_MECVV['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [767]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['MNA resultats'], SENSE_PA['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [768]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['MNA resultats'], SENSE_PA['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [994]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [995]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [997]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Valors de malnutrició

##### AMB_PA

In [769]:
norm_AMB_PA = AMB_PA['MNA resultats'].between(24, 30)
Percentatge_norm_AMB_PA = norm_AMB_PA.mean() * 100
num_observaciones_norm_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de norm_AMB_PA: {Percentatge_norm_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_norm_AMB_PA}")

Percentatge de norm_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [770]:
malnutr_AMB_PA = AMB_PA['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA = malnutr_AMB_PA.mean() * 100
num_observacions_malnutr_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de malnutr_AMB_PA: {Percentatge_malnutr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observacions_malnutr_AMB_PA}")

Percentatge de malnutr_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [771]:
malnut_AMB_PA = AMB_PA['MNA resultats'] < 17
Percentatge_malnut_AMB_PA = malnut_AMB_PA.mean() * 100
num_observacions_malnut_AMB_PA = malnut_AMB_PA.sum()

print(f"Percentatge de malnut_AMB_PA: {Percentatge_malnut_AMB_PA:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA: {num_observacions_malnut_AMB_PA}")

Percentatge de malnut_AMB_PA: 23.53%
Nombre d'observacions en malnut_AMB_PA: 4


##### AMB_PA_MECVV

In [772]:
norm_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'].between(24, 30)
Percentatge_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.mean() * 100
num_observacions_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de norm_AMB_PA_MECVV: {Percentatge_norm_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observacions_norm_AMB_PA_MECVV}")

Percentatge de norm_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [773]:
malnutr_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA_MECVV = malnutr_AMB_PA_MECVV.mean() * 100
num_observacions_malnutr_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de malnutr_AMB_PA_MECVV: {Percentatge_malnutr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observacions_malnutr_AMB_PA_MECVV}")

Percentatge de malnutr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [774]:
malnut_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'] < 17
Percentatge_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.mean() * 100
num_observacions_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.sum()

print(f"Percentatge de malnut_AMB_PA_MECVV: {Percentatge_malnut_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA_MECVV: {num_observacions_malnut_AMB_PA_MECVV}")

Percentatge de malnut_AMB_PA_MECVV: 25.00%
Nombre d'observacions en malnut_AMB_PA_MECVV: 2


##### SENSE_PA

In [775]:
norm_SENSE_PA = SENSE_PA['MNA resultats'].between(24, 30)
Percentatge_norm_SENSE_PA = norm_SENSE_PA.mean() * 100
num_observacions_norm_SENSE_PA = norm_SENSE_PA.sum()

print(f"Percentatge de norm_SENSE_PA: {Percentatge_norm_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en norm_SENSE_PA: {num_observacions_norm_SENSE_PA}")

Percentatge de norm_SENSE_PA: 0.00%
Nombre d'observacions en norm_SENSE_PA: 0


In [776]:
malnutr_SENSE_PA = SENSE_PA['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_SENSE_PA = malnutr_SENSE_PA.mean() * 100
num_observacions_malnutr_SENSE_PA = malnutr_SENSE_PA.sum()

print(f"Percentatge de malnutr_SENSE_PA: {Percentatge_malnutr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SENSE_PA: {num_observacions_malnutr_SENSE_PA}")

Percentatge de malnutr_SENSE_PA: 0.00%
Nombre d'observacions en malnutr_SENSE_PA: 0


In [777]:
malnut_SENSE_PA = SENSE_PA['MNA resultats'] < 17
Percentatge_malnut_SENSE_PA = malnut_SENSE_PA.mean() * 100
num_observacions_malnut_SENSE_PA = malnut_SENSE_PA.sum()

print(f"Percentatge de malnut_SENSE_PA: {Percentatge_malnut_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SENSE_PA: {num_observacions_malnut_SENSE_PA}")

Percentatge de malnut_SENSE_PA: 37.50%
Nombre d'observacions en malnut_SENSE_PA: 3


##### TOTAL

In [998]:
norm_TOTAL = TOTAL['MNA resultats'].between(24, 30)
Percentatge_norm_TOTAL = norm_TOTAL.mean() * 100
num_observacions_norm_TOTAL = norm_TOTAL.sum()

print(f"Percentatge de norm_TOTAL: {Percentatge_norm_TOTAL:.2f}%")
print(f"Nombre d'observacions en norm_TOTAL: {num_observacions_norm_TOTAL}")

Percentatge de norm_TOTAL: 0.00%
Nombre d'observacions en norm_TOTAL: 0


In [999]:
malnutr_TOTAL = TOTAL['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_TOTAL = malnutr_TOTAL.mean() * 100
num_observacions_malnutr_TOTAL = malnutr_TOTAL.sum()

print(f"Percentatge de malnutr_TOTAL: {Percentatge_malnutr_TOTAL:.2f}%")
print(f"Nombre d'observacions en malnutr_TOTAL: {num_observacions_malnutr_TOTAL}")

Percentatge de malnutr_TOTAL: 4.00%
Nombre d'observacions en malnutr_TOTAL: 2


In [1000]:
malnut_TOTAL = TOTAL['MNA resultats'] < 17
Percentatge_malnut_TOTAL = malnut_TOTAL.mean() * 100
num_observacions_malnut_TOTAL = malnut_TOTAL.sum()

print(f"Percentatge de malnut_TOTAL: {Percentatge_malnut_TOTAL:.2f}%")
print(f"Nombre d'observacions en malnut_TOTAL: {num_observacions_malnut_TOTAL}")

Percentatge de malnut_TOTAL: 18.00%
Nombre d'observacions en malnut_TOTAL: 9


#### Pvalor

##### Normal

In [778]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['MNA resultats'].between(24, 30), AMB_PA_MECVV['MNA resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV: 1.0


In [779]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['MNA resultats'].between(24, 30), SENSE_PA['MNA resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_SENSE_PA:", p_val_13)

P-valor entre norm_AMB_PA i norm_SENSE_PA: 1.0


In [780]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['MNA resultats'].between(24, 30), SENSE_PA['MNA resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA:", p_val_23)

P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA: 1.0


In [1001]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['MNA resultats'].between(24, 30), TOTAL['MNA resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_TOTAL:", p_val_14)

P-valor entre norm_AMB_PA i norm_TOTAL: 1.0


In [1002]:
t_stat_24, p_val_24 = mannwhitneyu(AMB_PA_MECVV['MNA resultats'].between(24, 30), TOTAL['MNA resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA_MECVV i norm_TOTAL:", p_val_24)

P-valor entre norm_AMB_PA_MECVV i norm_TOTAL: 1.0


In [1004]:
t_stat_34, p_val_34 = mannwhitneyu(SENSE_PA['MNA resultats'].between(24, 30), TOTAL['MNA resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_SENSE_PA i norm_TOTAL:", p_val_34)

P-valor entre norm_SENSE_PA i norm_TOTAL: 1.0


##### Risc de malnutrició

In [781]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['MNA resultats'].between(17, 23.5), AMB_PA_MECVV['MNA resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV:", p_val_12)

P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV: 1.0


In [782]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['MNA resultats'].between(17, 23.5), SENSE_PA['MNA resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA:", p_val_13)

P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA: 1.0


In [783]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['MNA resultats'].between(17, 23.5), SENSE_PA['MNA resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA:", p_val_23)

P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA: 1.0


##### Malnutrició

In [784]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['MNA resultats'] <17 , 
                                AMB_PA_MECVV['MNA resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV: 0.9686157491081862


In [785]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['MNA resultats'] <17 , 
                                SENSE_PA['MNA resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i malnut_SENSE_PA:", p_val_13)

P-valor entre malnut_AMB_PA i malnut_SENSE_PA: 0.5005446232443322


In [786]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['MNA resultats'] <17 , 
                                SENSE_PA['MNA resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA:", p_val_23)

P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA: 0.6477042192622259


## Index de l'EMINA

### Mitjana

In [787]:
Mitjana_emina = AMB_PA['EMINA_sumatorios_comparados'].mean()
desviacio_estandard_emina = AMB_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

KeyError: 'EMINA_sumatorios_comparados'

In [788]:
Mitjana_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacio_estandard_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

KeyError: 'EMINA_sumatorios_comparados'

In [789]:
Mitjana_emina = SENSE_PA['EMINA_sumatorios_comparados'].mean()
desviacio_estandard_emina = SENSE_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

KeyError: 'EMINA_sumatorios_comparados'

#### Pvalor

##### Normalitat

In [790]:
stat, p_value = shapiro(AMB_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'EMINA_sumatorios_comparados'

In [791]:
stat, p_value = shapiro(AMB_PA_MECVV['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'EMINA_sumatorios_comparados'

In [792]:
stat, p_value = shapiro(SENSE_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'EMINA_sumatorios_comparados'

In [793]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], AMB_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'EMINA_sumatorios_comparados'

In [794]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'EMINA_sumatorios_comparados'

In [795]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'EMINA_sumatorios_comparados'

### Valors EMINA

##### AMB_PA

In [796]:
lowr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA = lowr_AMB_PA.mean() * 100
num_observacions_lowr_AMB_PA = lowr_AMB_PA.sum()

print(f"Percentatge de lowr_AMB_PA: {Percentatge_lowr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA: {num_observacions_lowr_AMB_PA}")

KeyError: 'EMINA_sumatorios_comparados'

In [797]:
modr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA = modr_AMB_PA.mean() * 100
num_observacions_modr_AMB_PA = modr_AMB_PA.sum()

print(f"Percentatge de modr_AMB_PA: {Percentatge_modr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA: {num_observacions_modr_AMB_PA}")

KeyError: 'EMINA_sumatorios_comparados'

In [798]:
highr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA = highr_AMB_PA.mean() * 100
num_observacions_highr_AMB_PA = highr_AMB_PA.sum()

print(f"Percentatge de highr_AMB_PA: {Percentatge_highr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA: {num_observacions_highr_AMB_PA}")

KeyError: 'EMINA_sumatorios_comparados'

##### AMB_PA_MECVV

In [799]:
lowr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.mean() * 100
num_observacions_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.sum()

print(f"Percentatge de lowr_AMB_PA_MECVV: {Percentatge_lowr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA_MECVV: {num_observacions_lowr_AMB_PA_MECVV}")

KeyError: 'EMINA_sumatorios_comparados'

In [800]:
modr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.mean() * 100
num_observacions_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.sum()

print(f"Percentatge de modr_AMB_PA_MECVV: {Percentatge_modr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA_MECVV: {num_observacions_modr_AMB_PA_MECVV}")

KeyError: 'EMINA_sumatorios_comparados'

In [801]:
highr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.mean() * 100
num_observacions_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.sum()

print(f"Percentatge de highr_AMB_PA_MECVV: {Percentatge_highr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA_MECVV: {num_observacions_highr_AMB_PA_MECVV}")

KeyError: 'EMINA_sumatorios_comparados'

##### SENSE_PA

In [802]:
lowr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_SENSE_PA = lowr_SENSE_PA.mean() * 100
num_observacions_lowr_SENSE_PA = lowr_SENSE_PA.sum()

print(f"Percentatge de lowr_SENSE_PA: {Percentatge_lowr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SENSE_PA: {num_observacions_lowr_SENSE_PA}")

KeyError: 'EMINA_sumatorios_comparados'

In [803]:
modr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_SENSE_PA = modr_SENSE_PA.mean() * 100
num_observacions_modr_SENSE_PA = modr_SENSE_PA.sum()

print(f"Percentatge de modr_SENSE_PA: {Percentatge_modr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modr_SENSE_PA: {num_observacions_modr_SENSE_PA}")

KeyError: 'EMINA_sumatorios_comparados'

In [804]:
highr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_SENSE_PA = highr_SENSE_PA.mean() * 100
num_observacions_highr_SENSE_PA = highr_SENSE_PA.sum()

print(f"Percentatge de highr_SENSE_PA: {Percentatge_highr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en highr_SENSE_PA: {num_observacions_highr_SENSE_PA}")

KeyError: 'EMINA_sumatorios_comparados'

#### Pvalor 

##### Risc baix

In [805]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'EMINA_sumatorios_comparados'

In [806]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'EMINA_sumatorios_comparados'

In [807]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'EMINA_sumatorios_comparados'

##### Risc moderat

In [808]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'EMINA_sumatorios_comparados'

In [809]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'EMINA_sumatorios_comparados'

In [810]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'EMINA_sumatorios_comparados'

##### Risc alt

In [811]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'EMINA_sumatorios_comparados'

In [812]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'EMINA_sumatorios_comparados'

In [813]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'EMINA_sumatorios_comparados'

## Index de la canadenca 

### Mitjana

In [814]:
Mitjana_canad = AMB_PA['Canadenca_sumatorios_comparados'].mean()
desviacio_estandard_canad = AMB_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

KeyError: 'Canadenca_sumatorios_comparados'

In [815]:
Mitjana_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacio_estandard_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

KeyError: 'Canadenca_sumatorios_comparados'

In [816]:
Mitjana_canad = SENSE_PA['Canadenca_sumatorios_comparados'].mean()
desviacio_estandard_canad = SENSE_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

KeyError: 'Canadenca_sumatorios_comparados'

#### Pvalor

##### Normalitat

In [817]:
stat, p_value = shapiro(AMB_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Canadenca_sumatorios_comparados'

In [818]:
stat, p_value = shapiro(AMB_PA_MECVV['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Canadenca_sumatorios_comparados'

In [819]:
stat, p_value = shapiro(SENSE_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Canadenca_sumatorios_comparados'

In [820]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], AMB_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'Canadenca_sumatorios_comparados'

In [821]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'Canadenca_sumatorios_comparados'

In [822]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'Canadenca_sumatorios_comparados'

### Valors canadenca

##### AMB_PA 

In [823]:
mild_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA = mild_AMB_PA.mean() * 100
num_observacions_mild_AMB_PA = mild_AMB_PA.sum()

print(f"Percentatge de mild_AMB_PA: {Percentatge_mild_AMB_PA:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA: {num_observacions_mild_AMB_PA}")

KeyError: 'Canadenca_sumatorios_comparados'

In [824]:
modneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA = modneu_AMB_PA.mean() * 100
num_observacions_modneu_AMB_PA = modneu_AMB_PA.sum()

print(f"Percentatge de modneu_AMB_PA: {Percentatge_modneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA: {num_observacions_modneu_AMB_PA}")

KeyError: 'Canadenca_sumatorios_comparados'

In [825]:
sevneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA = sevneu_AMB_PA.mean() * 100
num_observacions_sevneu_AMB_PA = sevneu_AMB_PA.sum()

print(f"Percentatge de sevneu_AMB_PA: {Percentatge_sevneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA: {num_observacions_sevneu_AMB_PA}")

KeyError: 'Canadenca_sumatorios_comparados'

##### AMB_PA_MECVV

In [826]:
mild_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.mean() * 100
num_observacions_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.sum()

print(f"Percentatge de mild_AMB_PA_MECVV: {Percentatge_mild_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA_MECVV: {num_observacions_mild_AMB_PA_MECVV}")

KeyError: 'Canadenca_sumatorios_comparados'

In [827]:
modneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.mean() * 100
num_observacions_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.sum()

print(f"Percentatge de modneu_AMB_PA_MECVV: {Percentatge_modneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA_MECVV: {num_observacions_modneu_AMB_PA_MECVV}")

KeyError: 'Canadenca_sumatorios_comparados'

In [828]:
sevneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.mean() * 100
num_observacions_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.sum()

print(f"Percentatge de sevneu_AMB_PA_MECVV: {Percentatge_sevneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA_MECVV: {num_observacions_sevneu_AMB_PA_MECVV}")

KeyError: 'Canadenca_sumatorios_comparados'

##### SENSE_PA

In [829]:
mild_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_SENSE_PA = mild_SENSE_PA.mean() * 100
num_observacions_mild_SENSE_PA = mild_SENSE_PA.sum()

print(f"Percentatge de mild_SENSE_PA: {Percentatge_mild_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en mild_SENSE_PA: {num_observacions_mild_SENSE_PA}")

KeyError: 'Canadenca_sumatorios_comparados'

In [830]:
modneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_SENSE_PA = modneu_SENSE_PA.mean() * 100
num_observacions_modneu_SENSE_PA = modneu_SENSE_PA.sum()

print(f"Percentatge de modneu_SENSE_PA: {Percentatge_modneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SENSE_PA: {num_observacions_modneu_SENSE_PA}")

KeyError: 'Canadenca_sumatorios_comparados'

In [831]:
sevneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_SENSE_PA = sevneu_SENSE_PA.mean() * 100
num_observacions_sevneu_SENSE_PA = sevneu_SENSE_PA.sum()

print(f"Percentatge de sevneu_SENSE_PA: {Percentatge_sevneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SENSE_PA: {num_observacions_sevneu_SENSE_PA}")

KeyError: 'Canadenca_sumatorios_comparados'

#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [832]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'Canadenca_sumatorios_comparados'

In [833]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'Canadenca_sumatorios_comparados'

In [834]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'Canadenca_sumatorios_comparados'

##### Dèficit neurològic moderat

In [835]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'Canadenca_sumatorios_comparados'

In [836]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'Canadenca_sumatorios_comparados'

In [837]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'Canadenca_sumatorios_comparados'

##### Dèficit neurològic greu (sever)

In [838]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'Canadenca_sumatorios_comparados'

In [839]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'Canadenca_sumatorios_comparados'

In [840]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'Canadenca_sumatorios_comparados'

## Infeccions respiratòries

##### AMB_PA 

In [841]:
mainresp_AMB_PA= (AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA:.2f}%")

KeyError: 'Main_respiratory_infections_diagnosticada'

In [842]:
LRTI_AMB_PA= (AMB_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA:.2f}%")

KeyError: 'LRTI_diagnosticada'

In [843]:
COPD_AMB_PA= (AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA:.2f}%")

KeyError: 'COPD_exacerbations_diagnosticada'

In [844]:
fibro_AMB_PA= (AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA:.2f}%")

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

##### AMB_PA_MECVV

In [845]:
mainresp_AMB_PA_MECVV= (AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA_MECVV:.2f}%")

KeyError: 'Main_respiratory_infections_diagnosticada'

In [846]:
LRTI_AMB_PA_MECVV= (AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA_MECVV:.2f}%")

KeyError: 'LRTI_diagnosticada'

In [847]:
COPD_AMB_PA_MECVV = (AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA_MECVV:.2f}%")

KeyError: 'COPD_exacerbations_diagnosticada'

In [848]:
fibro_AMB_PA_MECVV = (AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA_MECVV:.2f}%")

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

##### SENSE_PA

In [849]:
mainresp_SENSE_PA= (SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SENSE_PA:.2f}%")

KeyError: 'Main_respiratory_infections_diagnosticada'

In [850]:
LRTI_SENSE_PA= (SENSE_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SENSE_PA:.2f}%")

KeyError: 'LRTI_diagnosticada'

In [851]:
COPD_SENSE_PA= (SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SENSE_PA:.2f}%")

KeyError: 'COPD_exacerbations_diagnosticada'

In [852]:
fibro_SENSE_PA= (SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SENSE_PA:.2f}%")

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

#### Pvalor

##### Normalitat

In [853]:
stat, p_value = shapiro(AMB_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Main_respiratory_infections_diagnosticada'

In [854]:
stat, p_value = shapiro(AMB_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'LRTI_diagnosticada'

In [855]:
stat, p_value = shapiro(AMB_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'COPD_exacerbations_diagnosticada'

In [856]:
stat, p_value = shapiro(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

In [857]:
stat, p_value = shapiro(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Main_respiratory_infections_diagnosticada'

In [858]:
stat, p_value = shapiro(AMB_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'LRTI_diagnosticada'

In [859]:
stat, p_value = shapiro(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'COPD_exacerbations_diagnosticada'

In [860]:
stat, p_value = shapiro(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

In [861]:
stat, p_value = shapiro(SENSE_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Main_respiratory_infections_diagnosticada'

In [862]:
stat, p_value = shapiro(SENSE_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'LRTI_diagnosticada'

In [863]:
stat, p_value = shapiro(SENSE_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'COPD_exacerbations_diagnosticada'

In [864]:
stat, p_value = shapiro(SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

##### Infeccions respiratòries principals 

In [865]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'Main_respiratory_infections_diagnosticada'

In [866]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'Main_respiratory_infections_diagnosticada'

In [867]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'Main_respiratory_infections_diagnosticada'

##### LRTI

In [868]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'LRTI_diagnosticada'

In [869]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'LRTI_diagnosticada'

In [870]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'LRTI_diagnosticada'

##### Exacerbacions de COPD

In [871]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'COPD_exacerbations_diagnosticada'

In [872]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'COPD_exacerbations_diagnosticada'

In [873]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'COPD_exacerbations_diagnosticada'

##### Fibrosi pulmonar i fibrotòrax

In [874]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

In [875]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

In [876]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'Pulmonary_fibrosis_fibrotorax_diagnosticada'

## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [877]:
AMB_PA.loc[:, 'albumina'] = pd.to_numeric(AMB_PA['albumina'], errors='coerce')

Mitjana_alb = AMB_PA['albumina'].mean()
desviacio_estandard_alb = AMB_PA['albumina'].std()

resultado = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

KeyError: 'albumina'

In [878]:
AMB_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(AMB_PA_MECVV['albumina'], errors='coerce')

Mitjana_alb = AMB_PA_MECVV['albumina'].mean()
desviacio_estandard_alb = AMB_PA_MECVV['albumina'].std()

resultado = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

KeyError: 'albumina'

In [879]:
SENSE_PA.loc[:, 'albumina'] = pd.to_numeric(SENSE_PA['albumina'], errors='coerce')

Mitjana_alb = SENSE_PA['albumina'].mean()
desviacio_estandard_alb = SENSE_PA['albumina'].std()

resultado = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

KeyError: 'albumina'

#### Pvalor

##### Normalitat

In [880]:
albpa = AMB_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'albumina'

In [881]:
albAMBpamecvv = AMB_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'albumina'

In [882]:
albsensepa= SENSE_PA['albumina'].dropna()
stat, p_value = shapiro(albsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'albumina'

In [883]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['albumina'], AMB_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'albumina'

In [884]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'albumina'

In [885]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'albumina'

### Proteïnes totals

In [886]:
AMB_PA.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA['proteinas totales'], errors='coerce')

Mitjana_prot = AMB_PA['proteinas totales'].mean()
desviacio_estandard_prot = AMB_PA['proteinas totales'].std()

resultado = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

KeyError: 'proteinas totales'

In [887]:
AMB_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA_MECVV['proteinas totales'], errors='coerce')

Mitjana_prot = AMB_PA_MECVV['proteinas totales'].mean()
desviacio_estandard_prot = AMB_PA_MECVV['proteinas totales'].std()

resultado = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

KeyError: 'proteinas totales'

In [888]:
SENSE_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SENSE_PA['proteinas totales'], errors='coerce')

Mitjana_prot = SENSE_PA['proteinas totales'].mean()
desviacio_estandard_prot = SENSE_PA['proteinas totales'].std()

resultado = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

KeyError: 'proteinas totales'

#### Pvalor

##### Normalitat

In [889]:
protAMBpa=AMB_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'proteinas totales'

In [890]:
protAMBpamecvv=AMB_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'proteinas totales'

In [891]:
protsensepa=SENSE_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'proteinas totales'

In [892]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['proteinas totales'], AMB_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'proteinas totales'

In [893]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'proteinas totales'

In [894]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'proteinas totales'

### Hemoglobina

In [895]:
AMB_PA.loc[:, 'Hb'] = pd.to_numeric(AMB_PA['Hb'], errors='coerce')

Mitjana_hb = AMB_PA['Hb'].mean()
desviacio_estandard_hb = AMB_PA['Hb'].std()

resultado = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: nan ± nan


In [896]:
AMB_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(AMB_PA_MECVV['Hb'], errors='coerce')

Mitjana_hb = AMB_PA_MECVV['Hb'].mean()
desviacio_estandard_hb = AMB_PA_MECVV['Hb'].std()

resultado = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: nan ± nan


In [897]:
SENSE_PA.loc[:, 'Hb'] = pd.to_numeric(SENSE_PA['Hb'], errors='coerce')

Mitjana_hb = SENSE_PA['Hb'].mean()
desviacio_estandard_hb = SENSE_PA['Hb'].std()

resultado = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: nan ± nan


##### Pvalor

##### Normalitat

In [898]:
hbAMBpa=AMB_PA['Hb'].dropna()
stat, p_value = shapiro(hbAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [899]:
hbAMBpamecvv=AMB_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [900]:
hbsensepa=SENSE_PA['Hb'].dropna()
stat, p_value = shapiro(hbsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [901]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Hb'], AMB_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [902]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [903]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Colesterol 

In [904]:
AMB_PA.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA['colesterol total'], errors='coerce')

Mitjana_col = AMB_PA['colesterol total'].mean()
desviacio_estandard_col = AMB_PA['colesterol total'].std()

resultado = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

KeyError: 'colesterol total'

In [905]:
AMB_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA_MECVV['colesterol total'], errors='coerce')

Mitjana_col = AMB_PA_MECVV['colesterol total'].mean()
desviacio_estandard_col = AMB_PA_MECVV['colesterol total'].std()

resultado = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

KeyError: 'colesterol total'

In [906]:
SENSE_PA.loc[:, 'colesterol total'] = pd.to_numeric(SENSE_PA['colesterol total'], errors='coerce')

Mitjana_col = SENSE_PA['colesterol total'].mean()
desviacio_estandard_col = SENSE_PA['colesterol total'].std()

resultado = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

KeyError: 'colesterol total'

#### Pvalor

##### Normalitat 



In [907]:
colAMBpa=AMB_PA['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'colesterol total'

In [908]:
colAMBpamecvv=AMB_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'colesterol total'

In [909]:
colsensepa=SENSE_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'colesterol total'

In [910]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['colesterol total'], AMB_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'colesterol total'

In [911]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'colesterol total'

In [912]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'colesterol total'

### Leucòcits

In [913]:
AMB_PA.loc[:, 'leucos'] = pd.to_numeric(AMB_PA['leucos'], errors='coerce')

Mitjana_leu = AMB_PA['leucos'].mean()
desviacio_estandard_leu = AMB_PA['leucos'].std()

resultado = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

KeyError: 'leucos'

In [914]:
AMB_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(AMB_PA_MECVV['leucos'], errors='coerce')

Mitjana_leu = AMB_PA_MECVV['leucos'].mean()
desviacio_estandard_leu = AMB_PA_MECVV['leucos'].std()

resultado = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

KeyError: 'leucos'

In [915]:
SENSE_PA.loc[:, 'leucos'] = pd.to_numeric(SENSE_PA['leucos'], errors='coerce')

Mitjana_leu = SENSE_PA['leucos'].mean()
desviacio_estandard_leu = SENSE_PA['leucos'].std()

resultado = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

KeyError: 'leucos'

#### Pvalor 

##### Normalitat

In [916]:
leuambpa=AMB_PA['leucos'].dropna()
stat, p_value = shapiro(leuambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'leucos'

In [917]:
leuambpamecvv=AMB_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'leucos'

In [918]:
leusensepa=SENSE_PA['leucos'].dropna()
stat, p_value = shapiro(leusensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'leucos'

In [919]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['leucos'], AMB_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'leucos'

In [920]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'leucos'

In [921]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'leucos'

### Limfòcits %

In [922]:
AMB_PA.loc[:, 'limfos'] = pd.to_numeric(AMB_PA['limfos'], errors='coerce')

Mitjana_limf = AMB_PA['limfos'].mean()
desviacio_estandard_limf = AMB_PA['limfos'].std()

resultado = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

KeyError: 'limfos'

In [923]:
AMB_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(AMB_PA_MECVV['limfos'], errors='coerce')

Mitjana_limf = AMB_PA_MECVV['limfos'].mean()
desviacio_estandard_limf = AMB_PA_MECVV['limfos'].std()

resultado = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

KeyError: 'limfos'

In [924]:
SENSE_PA.loc[:, 'limfos'] = pd.to_numeric(SENSE_PA['limfos'], errors='coerce')

Mitjana_limf = SENSE_PA['limfos'].mean()
desviacio_estandard_limf = SENSE_PA['limfos'].std()

resultado = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

KeyError: 'limfos'

#### Pvalor

##### Normalitat

In [925]:
limfambpa=AMB_PA['limfos'].dropna()
stat, p_value = shapiro(limfambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'limfos'

In [926]:
limfambpamecvv=AMB_PA_MECVV['limfos'].dropna()
stat, p_value = shapiro(limfambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'limfos'

In [927]:
limfsensepa=SENSE_PA['limfos'].dropna()
stat, p_value = shapiro(limfsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'limfos'

In [928]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['limfos'], AMB_PA_MECVV['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'limfos'

In [929]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['limfos'], SENSE_PA['limfos'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'limfos'

In [930]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['limfos'], SENSE_PA['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'limfos'

### Proteïna C Reactiva

In [931]:
AMB_PA.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA['prot C react'], errors='coerce')

Mitjana_protcr = AMB_PA['prot C react'].mean()
desviacio_estandard_protcr = AMB_PA['prot C react'].std()

resultado = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

KeyError: 'prot C react'

In [932]:
AMB_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA_MECVV['prot C react'], errors='coerce')

Mitjana_protcr = AMB_PA_MECVV['prot C react'].mean()
desviacio_estandard_protcr = AMB_PA_MECVV['prot C react'].std()

resultado = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

KeyError: 'prot C react'

In [933]:
SENSE_PA.loc[:, 'prot C react'] = pd.to_numeric(SENSE_PA['prot C react'], errors='coerce')

Mitjana_protcr = SENSE_PA['prot C react'].mean()
desviacio_estandard_protcr = SENSE_PA['prot C react'].std()

resultado = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

KeyError: 'prot C react'

#### Pvalor

##### Normalitat

In [934]:
pcrambpa=AMB_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'prot C react'

In [935]:
pcrambpamecvv=AMB_PA_MECVV['prot C react'].dropna()
stat, p_value = shapiro(pcrambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'prot C react'

In [936]:
pcrsensepa=SENSE_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'prot C react'

In [937]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['prot C react'], AMB_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'prot C react'

In [938]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'prot C react'

In [939]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'prot C react'

### Urea 

In [940]:
AMB_PA.loc[:, 'urea'] = pd.to_numeric(AMB_PA['urea'], errors='coerce')

Mitjana_urea = AMB_PA['urea'].mean()
desviacio_estandard_urea = AMB_PA['urea'].std()

resultado = f"{Mitjana_urea:.2f} ± {desviacio_estandard_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

KeyError: 'urea'

In [941]:
AMB_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(AMB_PA_MECVV['urea'], errors='coerce')

Mitjana_urea = AMB_PA_MECVV['urea'].mean()
desviacio_estandard_urea = AMB_PA_MECVV['urea'].std()

resultado = f"{Mitjana_urea:.2f} ± {desviacio_estandard_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

KeyError: 'urea'

In [942]:
SENSE_PA.loc[:, 'urea'] = pd.to_numeric(SENSE_PA['urea'], errors='coerce')

Mitjana_urea = SENSE_PA['urea'].mean()
desviacio_estandard_urea = SENSE_PA['urea'].std()

resultado = f"{Mitjana_urea:.2f} ± {desviacio_estandard_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

KeyError: 'urea'

#### Pvalor

##### Normalitat

In [943]:
uambpa=AMB_PA['urea'].dropna()
stat, p_value = shapiro(uambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'urea'

In [944]:
uambpamecvv=AMB_PA_MECVV['urea'].dropna()
stat, p_value = shapiro(uambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'urea'

In [945]:
usensepa=SENSE_PA['urea'].dropna()
stat, p_value = shapiro(usensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'urea'

In [946]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['urea'], AMB_PA_MECVV['urea'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

KeyError: 'urea'

In [947]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

KeyError: 'urea'

In [948]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

KeyError: 'urea'

### Filtrat glomerular CDK-EPI

In [949]:
AMB_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = AMB_PA['FGE CDK-EPI'].mean()
desviacio_estandard_fg = AMB_PA['FGE CDK-EPI'].std()

resultado = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: nan ± nan


In [950]:
AMB_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA_MECVV['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = AMB_PA_MECVV['FGE CDK-EPI'].mean()
desviacio_estandard_fg = AMB_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: nan ± nan


In [951]:
SENSE_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SENSE_PA['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = SENSE_PA['FGE CDK-EPI'].mean()
desviacio_estandard_fg = SENSE_PA['FGE CDK-EPI'].std()

resultado = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: nan ± nan


#### Pvalor

##### Normalitat

In [952]:
fgepiambpa=AMB_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [953]:
fgepiambpamecvv=AMB_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [954]:
fgepisensepa=SENSE_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [955]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], AMB_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [956]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [957]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular MDRD

In [958]:
AMB_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA['FGE MDRD'], errors='coerce')

Mitjana_fgmd = AMB_PA['FGE MDRD'].mean()
desviacio_estandard_fgmd = AMB_PA['FGE MDRD'].std()

resultado = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: nan ± nan


In [959]:
AMB_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA_MECVV['FGE MDRD'], errors='coerce')

Mitjana_fgmd = AMB_PA_MECVV['FGE MDRD'].mean()
desviacio_estandard_fgmd = AMB_PA_MECVV['FGE MDRD'].std()

resultado = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: nan ± nan


In [960]:
SENSE_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SENSE_PA['FGE MDRD'], errors='coerce')

Mitjana_fgmd = SENSE_PA['FGE MDRD'].mean()
desviacio_estandard_fgmd = SENSE_PA['FGE MDRD'].std()

resultado = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: nan ± nan


#### Pvalor

##### Normalitat

In [961]:
fgmd = AMB_PA['FGE MDRD'].dropna()
stat, p_value = shapiro(fgmd)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


ValueError: Data must be at least length 3.

In [962]:
stat, p_value = shapiro(AMB_PA_MECVV['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [963]:
stat, p_value = shapiro(SENSE_PA['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [964]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['FGE MDRD'], AMB_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [965]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [966]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i AMB_PA_MECVV:", p_val_23)

P-valor entre AMB_PA_MECVV i AMB_PA_MECVV: nan
